In [6]:
import sys
sys.path.append('..')

import socnet as sn
import networkx as nx

g1 = sn.load_graph('one-mode.gml')
g2 = sn.load_graph('two-mode.gml')

### Gráfico one-mode - Recomendações para melhor DJ

In [7]:
sn.show_graph(g1, nlab=True)

../socnet.py:59: RuntimeWarning:

divide by zero encountered in double_scalars

../socnet.py:187: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:188: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:189: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:190: RuntimeWarning:

invalid value encountered in double_scalars



In [8]:
sn.show_graph(g2, nlab=True)

## Indegree - Número de votos recebidos por Aluno

In [17]:
for n in g1.nodes:
    g1.nodes[n]['indegree'] = 0

indegree = {}
    
for n in g1.nodes:
    ind = 0
    for p in g1.predecessors(n):
        ind+=1
    g1.nodes[n]['indegree'] = ind
    indegree.update({g1.nodes[n]['label']:g1.nodes[n]['indegree']})

# ordenar em ordem decrescente
sorted_by_value = sorted(indegree.items(), key=lambda kv: kv[1],reverse=True)

for n in sorted_by_value:
    print(n[0] + ": " + str(n[1]))

Hugo: 12
Graicer: 10
Forba: 7
Toyama: 7
Fred: 7
Chen: 6
Rachel: 5
Vitoria: 5
Rapha: 3
Isa: 3
Antonio: 3
Leo: 2
Gabs: 2
Elisa: 2
Sabrina: 2
Rios: 2
Martim: 2
Jean: 1
Bruna: 1
Carlos: 1
Paulo: 0
Joao: 0
Pedro: 0
Tirta: 0
Ale: 0


## Medidas de centralidade para gŕafico two-mode

### Outdegree Ponderado Normalizado

In [19]:
for n in g2.nodes:
    g2.nodes[n]['indegree'] = 0
    g2.nodes[n]['outdegree_pond'] = 0
    g2.nodes[n]['indegree_pond'] = 0
    g2.nodes[n]['indegree_pond_norm'] = 0
    g2.nodes[n]['outdegree_pond_norm'] = 0
    g2.nodes[n]['g1_association'] = None
    
#indegree
for n in g2.nodes:
    
    count = 0
    for p in g2.predecessors(n):
        count+=1
    g2.nodes[n]['indegree'] = count

#outdegree ponderado
for n in g2.nodes:
    count = 0
    for p in g2.successors(n):
        count += g2.nodes[p]['indegree']    
    g2.nodes[n]['outdegree_pond'] = count
    
#indegree ponderado
total = 0
for n in g2.nodes:
    count = 0
    for p in g2.predecessors(n):
        count += g2.nodes[p]['outdegree_pond']    
    g2.nodes[n]['indegree_pond'] = count
    total += count
    
#indegree ponderado normalizado
totaln = 0
for n in g2.nodes:
    if g2.nodes[n]['indegree_pond'] != 0:
        g2.nodes[n]['indegree_pond_norm'] = g2.nodes[n]['indegree_pond']/total
        totaln += g2.nodes[n]['indegree_pond_norm']

#outdegree ponderado normalizado
for n in g2.nodes:
    count = 0
    for p in g2.successors(n):
        count += g2.nodes[p]['indegree_pond_norm']    
    g2.nodes[n]['outdegree_pond_norm'] = count



pr = {}
hubs = []

for n in g2.nodes:
    if g2.nodes[n]['outdegree_pond_norm'] != 0:
        hubs.append(n)

for hub in hubs:
    for n in g1.nodes:
        if g2.nodes[hub]['label'] == g1.nodes[n]['label']:
            g2.nodes[hub]['g1_association'] = n

for hub in hubs:
    pr.update({g2.nodes[hub]['g1_association']:g2.nodes[hub]['outdegree_pond_norm']})

pr.update({16:0})

# ordenar em ordem decrescente
sorted_by_value = sorted(pr.items(), key=lambda kv: kv[1],reverse=True)

for n in sorted_by_value:
    print(g1.nodes[n[0]]['label'] + ": " + str(n[1]))

Forba: 0.9329713721618953
Jean: 0.6860809476801579
Sabrina: 0.6768015794669299
Toyama: 0.5923988153998025
Rachel: 0.5393879565646594
Vitoria: 0.5296150049358341
Hugo: 0.5098716683119447
Isa: 0.5015794669299112
Graicer: 0.4670286278381047
Joao: 0.46584402764067134
Fred: 0.43731490621915103
Chen: 0.4340572556762093
Pedro: 0.41332675222112536
Martim: 0.4087857847976308
Rapha: 0.3862783810463968
Gabs: 0.3862783810463968
Ale: 0.37739387956564663
Bruna: 0.29842053307008887
Antonio: 0.2427443237907206
Paulo: 0.20829220138203358
Carlos: 0.20365251727541955
Elisa: 0.1531095755182626
Leo: 0.050345508390918066
Rios: 0


# Introdução: Quem é o melhor DJ?

Está análise busca descobrir quem seria o melhor DJ dentre os alunos da turma de Redes Sociais. Para isso, inicialmente, foi realizada uma votaçao, onde cada aluno poderia votar em outros colegas, os recomendando assim para serem DJ's. Ao fim, os dados foram compilados e se obteve um ranking. Porém, foi proposto uma outra maneira de se descobrir quem teria mais votos: Uma terceira métrica, que fizesse sentido em apontar o melhor DJ. Para isso, foram utilizados 2 grafos: O primeiro (one-mode) direcionado, onde os nós eram alunos e as arestas eram seus votos para outros colegas; Um segundo grafo (two-mode), onde existem 2 tipos de nós:

    -Alunos
    -Estilos Musicais
    
Os nós do tipo "Alunos" apontam para nós do tipo "Estilos Musicais", e a partir deste grafo, obtivemos medidas que indicavam o "Hub" de cada aluno, ou seja, sua capacidade de recomendar bem um estilo musical. A partir desta medida, começamos a pensar em qual métrica poderiamos utilizar.

### Terceira Métrica: Page Rank

Como terceira métrica, utilizaremos o PageRank, medida utilizada pelo Google para rankear as páginas de pesquisa. A importância de um nó específico, no PageRank, se da pela quantidade de nós que o citam, recomendam, etc, e pelo peso que cada um desses nós possuem. Ou seja, quanto mais nós apontando e quanto maior o peso desses nós, maior a importância do nó apontado.

Neste caso, iremos inicializar o peso de cada um dos nós com o valor de "Hub", pois para selecionarmos o melhor DJ, iremos nos basear pelo poder de recomendação de estilos musicais de cada aluno.
Assim, aplicamos a métrica PageRank em nossa rede, com os pesos iniciais aplicados, e veremos se quem possui o maior valor da centralidade é também o DJ mais bem votado no gráfico one-mode.

Abaixo, iremos comparar o ranqueamento feito em sala de aula com o ranqueamento feito pela análise da centralidade.

### Pagerank


In [21]:
pagerank = nx.pagerank(g1, nstart=pr)
sorted_by_value = sorted(pagerank.items(), key=lambda kv: kv[1],reverse=True)

for n in sorted_by_value:
    print(g1.nodes[n[0]]['label'] + ": " + str(n[1]))

Hugo: 0.16595597271454376
Graicer: 0.09627219473779855
Forba: 0.08924451484532504
Fred: 0.08659443113093473
Isa: 0.06871145747306044
Rachel: 0.06271494029242397
Toyama: 0.05872716189249575
Chen: 0.05632078101850835
Elisa: 0.053553135753432346
Vitoria: 0.03353628034482574
Sabrina: 0.029814608089819485
Rios: 0.021479125070811136
Rapha: 0.0206209392079862
Martim: 0.019199344299019542
Antonio: 0.013967063316954315
Bruna: 0.013856801854804014
Carlos: 0.011704897784264274
Leo: 0.00946919963610013
Gabs: 0.008362275120451082
Jean: 0.008160495889340867
Joao: 0.006730303210444701
Paulo: 0.006730303210444701
Ale: 0.006730303210444701
Pedro: 0.006730303210444701


## Votação Original

In [22]:
for n in g1.nodes:
    g1.nodes[n]['indegree'] = 0

indegree = {}
    
for n in g1.nodes:
    ind = 0
    for p in g1.predecessors(n):
        ind+=1
    g1.nodes[n]['indegree'] = ind
    indegree.update({g1.nodes[n]['label']:g1.nodes[n]['indegree']})

# ordenar em ordem decrescente
sorted_by_value = sorted(indegree.items(), key=lambda kv: kv[1],reverse=True)

for n in sorted_by_value:
    print(n[0] + ": " + str(n[1]))

Hugo: 12
Graicer: 10
Forba: 7
Toyama: 7
Fred: 7
Chen: 6
Rachel: 5
Vitoria: 5
Rapha: 3
Isa: 3
Antonio: 3
Leo: 2
Gabs: 2
Elisa: 2
Sabrina: 2
Rios: 2
Martim: 2
Jean: 1
Bruna: 1
Carlos: 1
Paulo: 0
Joao: 0
Pedro: 0
Tirta: 0
Ale: 0


## Resultados

Como se pode observar, o ranking feito através do uso de Hub e PageRank muito se assemelha ao rank da votação.Existem algumas mudanças nas posições, mas no geral os resultados iniciais da votação foram coerentes com os resultados da análise. Assim, pode-se concluir que o uso de PageRank e Hub para indicar preferências através de recomendações é viável.